In [6]:
import pandas as pd
import numpy as np
import json
import os

## Load Data

In [7]:
with open("data/product.json", "r") as file:
    raw_datas = json.load(file)
datas = pd.json_normalize(raw_datas)
datas.set_index('ProductID', inplace=True)

review = pd.read_csv('data/review.csv', index_col=0)
prediction = pd.read_csv('data/prediction.csv', index_col=0)
validation = pd.read_csv('data/validation.csv', index_col=0)

## Product Feature Extraction

In [8]:
print(f"total products: {len(datas)}")
datas.head()

total products: 6309


,category,tech1,description,fit,title,tech2,brand,feature,rank,main_cat,...,details.Publisher:,details.Publication Date:,details.Language:,details.ASIN:,details.Word Wise:,details.Lending:,details.Simultaneous Device Usage:,details.Release Date:,details.ISBN-10:,details.ISBN-13:
ProductID,,,,,,,,,,,,,,,,,,,,,
B000FBFMHU,"[Kindle Store, Kindle eBooks, Biographies & Me...",,[],,,,Visit Amazon's Frank W. Abagnale Page,[],"59,404 Paid in Kindle Store (",Buy a Kindle,...,"Broadway Books (November 19, 2002)","November 19, 2002",English,B000FBFMHU,Enabled,Enabled,NaN,NaN,NaN,NaN
B000FC27TA,"[Kindle Store, Kindle eBooks, Politics & Socia...",,[],,,,Visit Amazon's Karl Marx Page,[],"1,358,073 Paid in Kindle Store (",Buy a Kindle,...,"Bantam Classics; Reissue edition (August 31, 2...","August 31, 2004",English,B000FC27TA,Enabled,Not Enabled,NaN,NaN,NaN,NaN
B000FCKPG2,"[Kindle Store, Kindle eBooks, Romance]",,[],,,,Visit Amazon's Allison Brennan Page,[],"94,006 Paid in Kindle Store (",Buy a Kindle,...,"Ballantine Books (February 28, 2006)","February 28, 2006",English,B000FCKPG2,Enabled,Not Enabled,NaN,NaN,NaN,NaN
B000GCFWXW,"[Kindle Store, Kindle eBooks, Mystery, Thrille...",,[],,,,Visit Amazon's Lynsay Sands Page,[],"31,652 Paid in Kindle Store (",Buy a Kindle,...,"HarperCollins e-books (October 13, 2009)","October 13, 2009",English,B000GCFWXW,Enabled,Not Enabled,NaN,NaN,NaN,NaN
B000JMKRTI,"[Kindle Store, Kindle eBooks, Romance]",,[],,,,Visit Amazon's Fern Michaels Page,[],"1,031,468 Paid in Kindle Store (",Buy a Kindle,...,"Zebra (June 1, 2006)","June 1, 2006",English,B000JMKRTI,Enabled,Not Enabled,NaN,NaN,NaN,NaN


### Category

In [9]:
from langchain_ollama.llms import OllamaLLM

category_list = datas.explode('category')['category'].unique().tolist()

category_clusters = ["Fiction", "Non-Fiction", "Reference & Education",
                   "Arts & Entertainment", "Hobbies & Lifestyle", "Technology & Gaming"]

llm = OllamaLLM(model="llama3.1")

prompt_template = """
Please group the book category "{category}" into one of the following broader categories: {category_clusters}. You shall respond the broader category only.
"""

category_mapping = dict()

for cat in category_list:
    if "Kindle" in cat:
        category_mapping[cat] = "Invalid"
    else:
        prompt = prompt_template.format(
            category=cat,
            category_clusters=', '.join(
                f'{category}' for category in category_clusters)
        )
        cat_cluster = llm.invoke(prompt)
        category_mapping[cat] = cat_cluster if cat_cluster in category_clusters else "Invalid"
        
# Replace strings in the 'category' column using the category_mapping
feature_ranking = datas['category'].apply(
    lambda x: [category_mapping.get(item)
               for item in x if category_mapping.get(item) != "Invalid"]
)

# Explode the new_category list to create a row for each category
exploded_df = feature_ranking.explode()

# One-hot encode the exploded categories
one_hot_df = pd.get_dummies(exploded_df)

# Group by the original index and take the maximum to combine back into the original structure
feature_category = one_hot_df.groupby(exploded_df.index).max()

# Display the one-hot encoded DataFrame
print(f"One-hot encoded categories:")
print(f"shape: {len(datas)}")
feature_category.head()

One-hot encoded categories:
shape: 6309


,Arts & Entertainment,Fiction,Hobbies & Lifestyle,Non-Fiction,Reference & Education,Technology & Gaming
ProductID,,,,,,
B000FBFMHU,False,False,False,True,False,False
B000FC27TA,False,False,False,True,False,False
B000FCKPG2,True,False,False,False,False,False
B000GCFWXW,False,True,False,False,False,False
B000JMKRTI,True,False,False,False,False,False


In [11]:
from langchain_ollama.llms import OllamaLLM

filename = "category_mapping.json"

# Check if the file exists
if os.path.exists(filename):
    with open(filename, 'r') as json_file:
        category_mapping = json.load(json_file)

else:
    category_list = datas.explode('category')['category'].unique().tolist()

    category_clusters = ["Fiction", "Non-Fiction", "Reference & Education",
                        "Arts & Entertainment", "Hobbies & Lifestyle", "Technology & Gaming"]

    llm = OllamaLLM(model="llama3.1")

    prompt_template = """
    Please group the book category "{category}" into one of the following broader categories: {category_clusters}. You shall respond the broader category only.
    """

    category_mapping = dict()

    for cat in category_list:
        if "Kindle" in cat:
            category_mapping[cat] = "Invalid"
        else:
            prompt = prompt_template.format(
                category=cat,
                category_clusters=', '.join(
                    f'{category}' for category in category_clusters)
            )
            cat_cluster = llm.invoke(prompt)
            category_mapping[cat] = cat_cluster if cat_cluster in category_clusters else "Invalid"

    with open(filename, 'w') as json_file:
        json.dump(category_mapping, json_file, indent=4)

# Replace strings in the 'category' column using the category_mapping
feature_category = datas['category'].apply(
    lambda x: [category_mapping.get(item)
               for item in x if category_mapping.get(item) != "Invalid"]
)

# Explode the new_category list to create a row for each category
exploded_df = feature_category.explode()

# One-hot encode the exploded categories
one_hot_df = pd.get_dummies(exploded_df)

# Group by the original index and take the maximum to combine back into the original structure
feature_category = one_hot_df.groupby(exploded_df.index).max()

# Display the one-hot encoded DataFrame
print(f"One-hot encoded categories:")
print(f"shape: {len(datas)}")
feature_category.head()

One-hot encoded categories:
shape: 6309


,Arts & Entertainment,Fiction,Hobbies & Lifestyle,Non-Fiction,Reference & Education,Technology & Gaming
ProductID,,,,,,
B000FBFMHU,False,False,False,True,False,False
B000FC27TA,False,False,False,True,False,False
B000FCKPG2,True,False,False,False,False,False
B000GCFWXW,False,True,False,False,False,False
B000JMKRTI,True,False,False,False,False,False


### Language

In [6]:
print(f"Unique languages: {datas['details.Language:'].unique()}")

# One-hot encode the details.Language column
feature_language = pd.get_dummies(datas['details.Language:'])

# Display the one-hot encoded DataFrame
print(f"One-hot encoded language:")
print(f"shape: {len(feature_language)}")
feature_language.head()

Unique languages: ['English' 'Spanish']
One-hot encoded language:
shape: 6309


,English,Spanish
ProductID,,
B000FBFMHU,True,False
B000FC27TA,True,False
B000FCKPG2,True,False
B000GCFWXW,True,False
B000JMKRTI,True,False


### Ranking (Free / Paid)

In [16]:
# Extract Rank and Chart Type
datas['rank'] = datas['rank'].str.replace(',', '', regex=False)
pattern = r'(\d+) (Paid|Free) in Kindle Store \('
extracted = datas['rank'].str.extract(pattern)
extracted.columns = ['Rank', 'Type']

# Create a new DataFrame with the index from 'extracted'
feature_ranking = pd.DataFrame(index=extracted.index)

# Add "Paid" and "Free" columns filled with null values
feature_ranking['Paid'] = pd.NA
feature_ranking['Free'] = pd.NA

# Fill 'Paid' and 'Free' columns using boolean indexing
feature_ranking.loc[extracted['Type'] == 'Paid', 'Paid'] = extracted['Rank'].astype(float)
feature_ranking.loc[extracted['Type'] == 'Free', 'Free'] = extracted['Rank'].astype(float)

# Standardize data
feature_ranking['Paid'] = (feature_ranking['Paid'].max() - feature_ranking['Paid']) / feature_ranking['Paid'].max()
feature_ranking['Free'] = (feature_ranking['Free'].max() - feature_ranking['Free']) / feature_ranking['Free'].max()

# Fill empty data
feature_ranking.loc[extracted['Type'] == 'Paid', 'Free'] = 0
feature_ranking.loc[extracted['Type'] == 'Free', 'Paid'] = 0

# Display the one-hot encoded DataFrame
print(f"One-hot encoded language:")
print(f"shape: {len(feature_ranking)}")
feature_ranking.head()

One-hot encoded language:
shape: 6309


,Paid,Free
ProductID,,
B000FBFMHU,0.988578,0
B000FC27TA,0.738875,0
B000FCKPG2,0.981925,0
B000GCFWXW,0.993914,0
B000JMKRTI,0.801674,0


### Lending

In [8]:
print(f"Lending: {datas['details.Lending:'].unique()}")

# One-hot encode the details.Lending column
feature_lending = pd.DataFrame(index=extracted.index)
feature_lending['Lending'] = pd.NA
feature_lending.loc[datas['details.Lending:'] == 'Enabled'] = True
feature_lending.loc[datas['details.Lending:'] == 'Not Enabled'] = False

# Display the one-hot encoded DataFrame
print(f"One-hot encoded language:")
print(f"shape: {len(feature_lending)}")
feature_lending.head()

Lending: ['Enabled' 'Not Enabled' nan]
One-hot encoded language:
shape: 6309


,Lending
ProductID,
B000FBFMHU,True
B000FC27TA,False
B000FCKPG2,False
B000GCFWXW,False
B000JMKRTI,False


In [17]:
features = pd.DataFrame(index=datas.index)
features = features.merge(feature_category, on='ProductID')
features = features.merge(feature_language, on='ProductID')
features = features.merge(feature_ranking, on='ProductID')
features = features.merge(feature_lending, on='ProductID')
features

,Arts & Entertainment,Fiction,Hobbies & Lifestyle,Non-Fiction,Reference & Education,Technology & Gaming,English,Spanish,Paid,Free,Lending
ProductID,,,,,,,,,,,
B000FBFMHU,False,False,False,True,False,False,True,False,0.988578,0,True
B000FC27TA,False,False,False,True,False,False,True,False,0.738875,0,False
B000FCKPG2,True,False,False,False,False,False,True,False,0.981925,0,False
B000GCFWXW,False,True,False,False,False,False,True,False,0.993914,0,False
B000JMKRTI,True,False,False,False,False,False,True,False,0.801674,0,False
...,...,...,...,...,...,...,...,...,...,...,...
B01HFFPC2I,True,False,False,False,False,False,True,False,0.996575,0,True
B01HFGNGYI,True,False,False,False,False,False,True,False,0.957571,0,True
B01HFUF1GK,False,True,False,False,False,False,True,False,0.920718,0,True


In [36]:
features.to_csv('data/features.csv', index=True)

In [30]:
features.sum()

Arts & Entertainment            1582
Fiction                         4137
Hobbies & Lifestyle               98
Non-Fiction                      357
Reference & Education            131
Technology & Gaming               16
English                         6300
Spanish                            9
Paid                     4954.713251
Free                      788.445625
Lending                         4902
dtype: object

## Review Feature Extraction

In [114]:
review.iloc[np.random.randint(0, len(review))]

ProductID                                           B015QBDG3A
Text         It was a likeable story. Esther came off as th...
Summary                                                Not bad
Star                                                       4.0
Name: A2QM5QP4F68WLS, dtype: object

In [112]:
random_review = review.iloc[np.random.randint(0, len(review))]

print("##########Reviewer ID##########")
print(random_review.name)
print("\n##########Product ID##########")
print(random_review['ProductID'])
print("\n##########Text##########")
print(random_review["Text"])
print("\n##########Summary##########")
print(random_review["Summary"])
print("\n##########Star##########")
print(random_review["Star"])

##########Reviewer ID##########
AC0CW3OQUS1ZJ

##########Product ID##########
B00OM8A948

##########Text##########
It just goes along. No part faster or slower, no ups or downs, just step 1,2,3etc. You keep hearing about duty and responsibility, but yet what about love and her heart. While well written and clean, it just is. Overall a nice read, but not too exciting.

##########Summary##########
Good but

##########Star##########
3.0


In [25]:
print(f"total review: {len(review)}")
review.head()

total review: 52512


,ProductID,Text,Summary,Star
ReviewerID,,,,
A1XJXYKOWCH9XT,B000FBFMHU,Liked the movie. Loved the book. It really giv...,Liked the movie. Loved the book!,5.0
A1K4S4MWXI9E9M,B000FC27TA,Purchased more out of curiosity than any real ...,"Not my favorite, but...",3.0
A3LF914GG87TWP,B000FC27TA,"I actually received this text as an ebook, sin...",An interesting read,4.0
A1CNQTCRQ35IMM,B000FCKPG2,REVIEWER'S OPINION:\nThis was labeled as roman...,This was labeled romance but there was less ro...,2.0
AU510CVD9XDG,B000GCFWXW,I have been saving the Argeneau novels for awh...,Science Fiction not Paranormal Romance,2.0


In [27]:
review_extended = review.reset_index().merge(features, on='ProductID', how='right')
review_extended.head()

,ReviewerID,ProductID,Text,Summary,Star,Arts & Entertainment,Fiction,Hobbies & Lifestyle,Non-Fiction,Reference & Education,Technology & Gaming,English,Spanish,Paid,Free,Lending
0,A1XJXYKOWCH9XT,B000FBFMHU,Liked the movie. Loved the book. It really giv...,Liked the movie. Loved the book!,5.0,False,False,False,True,False,False,True,False,0.988578,0,True
1,A1K4S4MWXI9E9M,B000FC27TA,Purchased more out of curiosity than any real ...,"Not my favorite, but...",3.0,False,False,False,True,False,False,True,False,0.738875,0,False
2,A3LF914GG87TWP,B000FC27TA,"I actually received this text as an ebook, sin...",An interesting read,4.0,False,False,False,True,False,False,True,False,0.738875,0,False
3,A1CNQTCRQ35IMM,B000FCKPG2,REVIEWER'S OPINION:\nThis was labeled as roman...,This was labeled romance but there was less ro...,2.0,True,False,False,False,False,False,True,False,0.981925,0,False
4,AU510CVD9XDG,B000GCFWXW,I have been saving the Argeneau novels for awh...,Science Fiction not Paranormal Romance,2.0,False,True,False,False,False,False,True,False,0.993914,0,False


In [32]:
top_products = review_extended['ProductID'].value_counts().sort_values(ascending=False).head(5)
top_products

ProductID
B00YN6XHMU    86
B00R04OXC0    67
B00MZWA6KA    67
B00R04OX5C    64
B00JTCH5RQ    64
Name: count, dtype: int64

##

In [35]:
# x = review_extended.loc[review_extended["ReviewerID"] == "A25X28UZCW2J6G"].sort_values(by='Star', ascending=True)
x = review_extended.loc[review_extended["ProductID"] == "B00YN6XHMU"].sort_values(by='Star', ascending=True)
print(x.mean(numeric_only=True))
x

Star                     4.22093
Arts & Entertainment     0.00000
Fiction                  1.00000
Hobbies & Lifestyle      0.00000
Non-Fiction              0.00000
Reference & Education    0.00000
Technology & Gaming      0.00000
English                  1.00000
Spanish                  0.00000
dtype: float64


,ReviewerID,ProductID,Text,Summary,Star,Arts & Entertainment,Fiction,Hobbies & Lifestyle,Non-Fiction,Reference & Education,Technology & Gaming,English,Spanish,Paid,Free,Lending
37601,A1N4DZQD6XINNY,B00YN6XHMU,What did I just read!? Seriously! I am a Fifty...,UNHAPPY!!!,1.0,False,True,False,False,False,False,True,False,0.97932,0,False
37547,A2Y3ZGVRA3S23L,B00YN6XHMU,The fascination for this series started in the...,Should Have Listened to my Guts and Stayed Awa...,1.0,False,True,False,False,False,False,True,False,0.97932,0,False
37562,A190TSUIRJIFUK,B00YN6XHMU,As talented as authors are you come across s...,Seriously?,1.0,False,True,False,False,False,False,True,False,0.97932,0,False
37533,A16ZTSR8M5J5G2,B00YN6XHMU,At least in this book we're not subjected to A...,I got this for 99 cents and I still overpaid,1.0,False,True,False,False,False,False,True,False,0.97932,0,False
37530,A179ZGXWA4RW3Q,B00YN6XHMU,Oh my EL James! You outdid yourself with this ...,If you want to hate CG read this book.,1.0,False,True,False,False,False,False,True,False,0.97932,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37567,A1FUH1O6FCTUYG,B00YN6XHMU,>Review Grey -\n\n>I am an independent review...,Best book yet,5.0,False,True,False,False,False,False,True,False,0.97932,0,False
37527,A143XY6COA69A,B00YN6XHMU,Wow I'm blown away again by E L James. I loved...,so great,5.0,False,True,False,False,False,False,True,False,0.97932,0,False
37569,A3BBA9OF0GNM0A,B00YN6XHMU,Just like everyone else I was so excited to fi...,Love Christian!!!,5.0,False,True,False,False,False,False,True,False,0.97932,0,False
37557,A1QJ17BR5K3C8K,B00YN6XHMU,Let me just say that I was thrilled when I fou...,I Love It,5.0,False,True,False,False,False,False,True,False,0.97932,0,False


In [6]:
prediction.head()

,ProductID,Star
ReviewerID,,
A2MK1L1Y74WTWH,B01GT5XDFS,0
A19I68RW4PBT29,B00OME9OQQ,0
A1UPHTDW5GM12T,B01GSRNLOK,0
A1LFIFPYMOJ8RV,B01CUJYMR0,0
A10Y597K071WTQ,B004SI455Q,0


In [7]:
validation.head()

,ProductID,Star
ReviewerID,,
A25X28UZCW2J6G,B00K9V6B94,4.0
A1FUH1O6FCTUYG,B00GZANS6M,5.0
AAUVEEG5YLZAX,B01864DDVO,5.0
A3VQLGTYTL5196,B001BXNQ2O,5.0
A10JAUCIGVRW9F,B0116MZUS2,5.0
